# Data Analysis and Visualization in Python
## Combining DataFrames with pandas
Questions
* Can I work with data from multiple sources?
* How can I combine data from different data sets?

Objectives
* Combine data from multiple files into a single DataFrame using `concat` and `merge`.
* Combine two DataFrames using a unique ID found in both DataFrames.

## Loading our data

In [ ]:
# First make sure pandas is loaded
import pandas as pd

# Read in the survey csv
surveys_df = pd.read_csv('../data/surveys.csv')

## Concatenating DataFrames

In [ ]:
# Read in first 10 lines of surveys table
surveys_head10 = surveys_df.head(10)
surveys_head10

In [ ]:
# Grab the last 10 rows
surveys_tail10 = surveys_df.tail(10)
surveys_tail10

In [ ]:
# Stack the DataFrames on top of each other
df_list = [surveys_head10, surveys_tail10]
vertical_stack = pd.concat(df_list, axis='index')
vertical_stack

In [ ]:
# Reset index values of the dataframe
# The drop=True option avoids adding new index column with old index values
vertical_stack = vertical_stack.reset_index(drop=True)
vertical_stack

### Writing Out Data to CSV

In [ ]:
# Write DataFrame to CSV without the index
csv_file = 'surveys_sub.csv'
vertical_stack.to_csv(csv_file, index=False)

In [ ]:
# Read our output back into python and make sure all looks good
new_output = pd.read_csv(csv_file)
new_output

## Exercise - Concatenating DataFrames
* In `surveys_df`, select rows where the year is 2001.
  Do the same for year 2002.
* Concatenate both dataframes.

(3 min.)

In [ ]:
# Get data for each year
survey2001 = surveys_df[surveys_df['year'] ###]
survey2002 = surveys_df[surveys_df['year'] ###]

# Concatenate vertically
survey_all = ###

* Compute the average weight by sex for each year. (1 min.)

In [ ]:
# Get the average weight by sex for each year
weight_year = survey_all.groupby(['year', 'sex'])###
weight_year = weight_year.unstack()
weight_year

* Export your results as a CSV and make sure
  it reads back into python properly. (2 min.)

In [ ]:
# Writing to file while keeping the index
csv_file = 'weight_for_year.csv'
weight_year###

# Reading it back in with a specified index column
pd.read_csv(csv_file, index_col=###)

## Joining Two DataFrames

In [ ]:
# Import a small subset of the species data designed for this part of the lesson
species_sub = pd.read_csv('../data/speciesSubset.csv')
species_sub

### Identifying join keys

In [ ]:
surveys_head10.columns

In [ ]:
species_sub.columns

### Inner joins

![Inner join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/inner-join.png)

In [ ]:
# Computing the inner join of surveys_head10 and species_sub
key = 'species_id'
merged_inner = pd.merge(left=surveys_head10, right=species_sub,
                        left_on=key, right_on=key)
# What's the size of the output data?
merged_inner.shape

In [ ]:
merged_inner

### Left joins

![Left join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/left-join.png)

In [ ]:
merged_left = pd.merge(left=surveys_head10, right=species_sub,
                       on=key, how='left')
# What's the size of the output data?
merged_left.shape

In [ ]:
merged_left

### Other join types
* `how='right'` : all rows from the right DataFrame are kept
* `how='outer'` : all pairwise combinations of rows from both DataFrames

## Exercise - Joining all data
`1`. Create a new DataFrame by joining the contents of the
`surveys.csv` and `species.csv` tables. Keep all survey records.
(3 min.)

In [ ]:
species_df = pd.read_csv('../data/species.csv')
key = 'species_id'

merged_left = pd.merge(
    left=surveys_df, right=###, on=###, how=###)
merged_left.shape

`2`. Calculate and plot the distribution of surveys (i.e. the
number of `record_id`) by `taxa` for each `plot_id`. (3 min.)

In [ ]:
by_site_taxa = merged_left###
taxa_site = by_site_taxa['record_id']###
taxa_site.tail()

In [ ]:
taxa_site.plot(kind='bar', stacked=True)

`3`. Calculate and plot the distribution
of `taxa` by `sex` for each `plot_id`. (2 min.)

In [ ]:
# Data cleanup
merged_left['sex'] = merged_left['sex'].fillna('F|M')
invalid_mask = ~merged_left['sex'].isin(['F', 'F|M', 'M'])
merged_left.loc[invalid_mask, 'sex'] = "F|M"

In [ ]:
ntaxa_sex_site = merged_left.groupby(
    ['plot_id', 'sex'])[###].nunique()#.reset_index(level=1)
ntaxa_sex_site.tail()

In [ ]:
# Use pivot_table() instead of unstack()
pivot_taxa_sex_site = ntaxa_sex_site.pivot_table(
    values='taxa', columns='sex', index=ntaxa_sex_site.index)
pivot_taxa_sex_site.head()

In [ ]:
pivot_taxa_sex_site.plot(kind="bar", stacked=True)

## Technical Summary
* **Concatenate** DataFrames with `pandas.concat()`
  * Requires a list of DataFrames
  * Vertically if `axis='index'` (by default)
  * Horizontally if `axis='columns'`
  * Resetting the index: `reset_index(drop=True)`
* **Joining** DataFrmaes with `pandas.merge()`
  * `left=`, `right=`: both DataFrames to join
  * `left_on=`, `right_on=`: join key for each DataFrame
  * `on=`: join key for both DataFrames
  * `how=`: `'inner'` (default), `'left'`, `'right'`, `'outer'`
* **Pivot table**  `pivot_table()`
  * `values=colX`
  * `index=[col_ind]`
  * `columns=[category1, category2]`
  * `aggfunc=numpy.mean` (default: mean)